In [328]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [329]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [330]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [331]:
%%capture pwd
!pwd

In [332]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-has-the-pandemic-affected-the-mental-health-of-healthcare-workers',
 300,
 500)

# Fig 1

In [333]:
# read stata .gph graph data. prefer to use .dta - since .gph is not natively loaded into pandas
# use this only a visual aid to fill out actual data table
fs1 = open("raw/Fig1.gph", "r", encoding="latin1").read()
sersets = [i.split("<EndSerset>")[0] for i in fs1.split("<BeginSerset>")[1:]]
data = []
for j in range(len(sersets)):
    s = sersets[j]
    s1 = [i.split("<EndSeries>")[0] for i in s.split("<BeginSeries>")[1:]]
    mins = [float(i.split("min =  ")[1].split("\n.max")[0]) for i in s1]
    maxs = [float(i.split("max =  ")[1].split("\n.med")[0]) for i in s1]
    names = [i.split('name = `"')[1].split('"')[0] for i in s1]
    df = pd.DataFrame(np.array([mins, maxs, names]).T, columns=["min", "max", "name"])
    df["id"] = j
    data.append(df)
df = pd.concat(data)
df["name"] = (
    df["name"]
    .replace("__000006", "bar")
    .replace("__00000D", "maxbar")
    .replace("__00000C", "minbar")
    .replace("__000003", "x")
)
df.set_index(["id", "name"]).stack().reset_index().set_index(
    ["id", "level_2", "name"]
).unstack()[0].drop_duplicates()

name                      bar             maxbar             minbar  \
id level_2                                                            
0  max      24.61820411682129  26.70649909973145  22.52990913391113   
   min               11.71875  14.50863552093506  8.928864479064941   
2  max      28.17502593994141  31.05848693847656  25.29156684875488   
   min      13.97738933563232  16.15845108032227  11.79632759094238   
4  max      20.09774017333984  22.04034042358398   18.1551399230957   
   min              13.671875  16.65177536010742  10.69197463989258   
6  max      21.66488838195801  24.30547523498535  19.02430152893066   
   min      15.93011283874512  18.23196983337402  13.62825584411621   

name                        x  
id level_2                     
0  max      2.799999952316284  
   min      0.800000011920929  
2  max      3.200000047683716  
   min      1.200000047683716  
4  max      2.799999952316284  
   min      0.800000011920929  
6  max      3.200000047683716  
   min      1.200000047683716

In [334]:
df = pd.read_excel("raw/Book1.xlsx")
df["Time"] = (
    df["Time"]
    .astype(str)
    .str[:7]
    .replace("2020-06", " Jun 2020")
    .str.replace("Dec", "Dec 2020")
    .str.strip()
)

## a

In [335]:
f = "fig1a_anxiety"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,Anxiety,Jun 2020,Catalonia,15.9,13.6,18.2
1,Anxiety,Nov/Dec 2020,Catalonia,14.0,11.8,16.2
2,Anxiety,Jun 2020,Italy,24.6,22.6,26.7
3,Anxiety,Nov/Dec 2020,Italy,28.2,25.3,31.1
4,Anxiety,Jun 2020,UK,11.7,8.9,14.5


In [336]:
base = (
    alt.Chart(f1a)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Anxiety"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+21,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

## b

In [337]:
f = "fig1b_depression"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,Anxiety,Jun 2020,Catalonia,15.9,13.6,18.2
1,Anxiety,Nov/Dec 2020,Catalonia,14.0,11.8,16.2
2,Anxiety,Jun 2020,Italy,24.6,22.6,26.7
3,Anxiety,Nov/Dec 2020,Italy,28.2,25.3,31.1
4,Anxiety,Jun 2020,UK,11.7,8.9,14.5


In [338]:
base = (
    alt.Chart(f1b)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Depression"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+21,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [339]:
# read stata .gph graph data. prefer to use .dta - since .gph is not natively loaded into pandas
# use this only a visual aid to fill out actual data table
fs1 = open("raw/Fig2.gph", "r", encoding="latin1").read()
sersets = [i.split("<EndSerset>")[0] for i in fs1.split("<BeginSerset>")[1:]]
data = []
for j in range(len(sersets)):
    s = sersets[j]
    s1 = [i.split("<EndSeries>")[0] for i in s.split("<BeginSeries>")[1:]]
    mins = [float(i.split("min =  ")[1].split("\n.max")[0]) for i in s1]
    maxs = [float(i.split("max =  ")[1].split("\n.med")[0]) for i in s1]
    names = [i.split('name = `"')[1].split('"')[0] for i in s1]
    df = pd.DataFrame(np.array([mins, maxs, names]).T, columns=["min", "max", "name"])
    df["id"] = j
    data.append(df)
df = pd.concat(data)
df["name"] = (
    df["name"]
    .replace("__000006", "bar")
    .replace("__00000D", "maxbar")
    .replace("__00000C", "minbar")
    .replace("__000003", "x")
)
df.set_index(["id", "name"]).stack().reset_index().set_index(
    ["id", "level_2", "name"]
).unstack()[0].drop_duplicates()

name                      bar             maxbar             minbar  \
id level_2                                                            
0  max      19.71080589294434  21.86360931396484  17.55800247192383   
   min      10.20408153533936  12.55773639678955  7.850427150726318   
2  max      32.38095092773438  34.96681976318359  29.79508590698242   
   min      17.32673263549805  19.45932769775391  13.71637153625488   
4  max      15.29680347442627  17.24473571777344   13.3488712310791   
   min      12.08791255950928   14.6226167678833  9.553207397460938   
6  max      26.26984214782715  28.70192337036133  23.83775901794434   
   min      18.97689819335938  21.18634986877441  16.38217926025391   
8  max      29.27161407470703  31.59999847412109  26.94322776794434   
   min      15.56802272796631  18.23198318481445  12.90406131744385   
10 max      21.44796371459961  23.87085914611816  19.02506828308105   
   min      7.194244384765625  11.50832176208496  2.880167722702026   
12 max       23.8938045501709  26.07596778869629  21.71164131164551   
   min      18.23281860351563  20.89893913269043  15.39572334289551   
14 max      16.38009071350098  18.56471061706543  14.19547080993652   
   min      5.755395889282227  9.643819808959961  1.866971254348755   

name                        x  
id level_2                     
0  max      2.799999952316284  
   min      0.800000011920929  
2  max      3.200000047683716  
   min      1.200000047683716  
4  max      2.799999952316284  
   min      0.800000011920929  
6  max      3.200000047683716  
   min      1.200000047683716  
8  max      2.799999952316284  
   min      0.800000011920929  
10 max      3.200000047683716  
   min      1.200000047683716  
12 max      2.799999952316284  
   min      0.800000011920929  
14 max      3.200000047683716  
   min      1.200000047683716

In [340]:
df = pd.read_excel("raw/Book2.xlsx")
df["Time"] = (
    df["Time"]
    .astype(str)
    .str[:7]
    .replace("2020-06", " Jun 2020")
    .str.replace("Dec", "Dec 2020")
    .str.strip()
)

## a

In [341]:
f = "fig2a_anxiety_sex"
f2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Type,Country,Value,Min,Max
0,Anxiety,Men,Sex,Catalonia,10.2,7.9,12.5
1,Anxiety,Women,Sex,Catalonia,17.3,15.1,19.5
2,Anxiety,Men,Sex,Italy,19.7,17.5,21.9
3,Anxiety,Women,Sex,Italy,32.4,29.8,35.0
4,Anxiety,Men,Sex,UK,12.3,14.6,10.0


In [342]:
base = (
    alt.Chart(f2a)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Anxiety"')
    .transform_filter('datum.Type=="Sex"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="Men"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time=="Women"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+35,
        dy=14,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## b

In [343]:
f = "fig2b_depression_sex"
f2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Type,Country,Value,Min,Max
0,Anxiety,Men,Sex,Catalonia,10.2,7.9,12.5
1,Anxiety,Women,Sex,Catalonia,17.3,15.1,19.5
2,Anxiety,Men,Sex,Italy,19.7,17.5,21.9
3,Anxiety,Women,Sex,Italy,32.4,29.8,35.0
4,Anxiety,Men,Sex,UK,12.3,14.6,10.0


In [344]:
base = (
    alt.Chart(f2b)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Depression"')
    .transform_filter('datum.Type=="Sex"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="Men"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time=="Women"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+21,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## c

In [345]:
f = "fig2c_anxiety_age"
f2c = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2c.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2c = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Type,Country,Value,Min,Max
0,Anxiety,Men,Sex,Catalonia,10.2,7.9,12.5
1,Anxiety,Women,Sex,Catalonia,17.3,15.1,19.5
2,Anxiety,Men,Sex,Italy,19.7,17.5,21.9
3,Anxiety,Women,Sex,Italy,32.4,29.8,35.0
4,Anxiety,Men,Sex,UK,12.3,14.6,10.0


In [346]:
base = (
    alt.Chart(f2c)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Anxiety"')
    .transform_filter('datum.Type=="Age"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="< 60"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="< 60"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+23,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## d

In [347]:
f = "fig2d_depression_age"
f2d = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2d.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2d = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Type,Country,Value,Min,Max
0,Anxiety,Men,Sex,Catalonia,10.2,7.9,12.5
1,Anxiety,Women,Sex,Catalonia,17.3,15.1,19.5
2,Anxiety,Men,Sex,Italy,19.7,17.5,21.9
3,Anxiety,Women,Sex,Italy,32.4,29.8,35.0
4,Anxiety,Men,Sex,UK,12.3,14.6,10.0


In [348]:
base = (
    alt.Chart(f2d)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="Depression"')
    .transform_filter('datum.Type=="Age"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=40,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 35]),
        )
    )
    .transform_filter('datum.Time=="< 60"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="< 60"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+23,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [349]:
# read stata .gph graph data. prefer to use .dta - since .gph is not natively loaded into pandas
# use this only a visual aid to fill out actual data table
fs1 = open("raw/Fig3.gph", "r", encoding="latin1").read()
sersets = [i.split("<EndSerset>")[0] for i in fs1.split("<BeginSerset>")[1:]]
data = []
for j in range(len(sersets)):
    s = sersets[j]
    s1 = [i.split("<EndSeries>")[0] for i in s.split("<BeginSeries>")[1:]]
    mins = [float(i.split("min =  ")[1].split("\n.max")[0]) for i in s1]
    maxs = [float(i.split("max =  ")[1].split("\n.med")[0]) for i in s1]
    names = [i.split('name = `"')[1].split('"')[0] for i in s1]
    df = pd.DataFrame(np.array([mins, maxs, names]).T, columns=["min", "max", "name"])
    df["id"] = j
    data.append(df)
df = pd.concat(data)
df["name"] = (
    df["name"]
    .replace("__000006", "bar")
    .replace("__00000D", "maxbar")
    .replace("__00000C", "minbar")
    .replace("__000003", "x")
)
df.set_index(["id", "name"]).stack().reset_index().set_index(
    ["id", "level_2", "name"]
).unstack()[0].drop_duplicates().sort_values("bar")

,name,bar,maxbar,minbar,x
id,level_2,,,,
2,min,10.06097602844238,13.32320594787598,6.798745155334473,1.200000047683716
0,min,16.14906883239746,19.43558311462402,12.8625545501709,0.800000011920929
14,min,21.31868171691895,23.98265266418457,18.65471076965332,1.200000047683716
4,min,22.56728744506836,26.30073356628418,18.83384323120117,0.800000011920929
12,min,22.85368728637695,25.71837615966797,19.98900032043457,0.800000011920929
8,min,24.98453903198242,27.09615516662598,22.87292289733887,0.800000011920929
6,min,26.42543792724609,29.29034233093262,21.73682403564453,1.200000047683716
2,max,30.1339282989502,33.14169311523438,27.12616348266602,3.200000047683716
8,max,30.15872955322266,34.17111968994141,26.57784271240234,2.799999952316284


In [350]:
df = pd.read_excel("raw/Book3.xlsx")
df["Time"] = (
    df["Time"]
    .astype(str)
    .str[:7]
    .replace("2020-06", " Jun 2020")
    .str.replace("Dec", "Dec 2020")
    .str.strip()
)

## a

In [351]:
f = "fig3a_ppe"
f3a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,A,Jun 2020,Catalonia,25.8,22.6,29.0
1,A,Nov/Dec 2020,Catalonia,15.4,13.8,17.0
2,A,Jun 2020,Italy,50.1,47.6,52.6
3,A,Nov/Dec 2020,Italy,30.1,27.1,33.1
4,A,Jun 2020,UK,16.1,12.8,19.4


In [352]:
base = (
    alt.Chart(f3a)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="A"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=20,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 65]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+22,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## b

In [353]:
f = "fig3b_vulnerable"
f3b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,A,Jun 2020,Catalonia,25.8,22.6,29.0
1,A,Nov/Dec 2020,Catalonia,15.4,13.8,17.0
2,A,Jun 2020,Italy,50.1,47.6,52.6
3,A,Nov/Dec 2020,Italy,30.1,27.1,33.1
4,A,Jun 2020,UK,16.1,12.8,19.4


In [354]:
base = (
    alt.Chart(f3b)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="B"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=20,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 65]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+22,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## c

In [355]:
f = "fig3c_directly"
f3c = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3c.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3c = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,A,Jun 2020,Catalonia,25.8,22.6,29.0
1,A,Nov/Dec 2020,Catalonia,15.4,13.8,17.0
2,A,Jun 2020,Italy,50.1,47.6,52.6
3,A,Nov/Dec 2020,Italy,30.1,27.1,33.1
4,A,Jun 2020,UK,16.1,12.8,19.4


In [356]:
base = (
    alt.Chart(f3c)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="C"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=20,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 65]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="left",
        fontWeight="bold",
        dx=+37,
        dy=14,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## d

In [357]:
f = "fig3d_death"
f3d = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3d.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3d = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Category,Time,Country,Value,Min,Max
0,A,Jun 2020,Catalonia,25.8,22.6,29.0
1,A,Nov/Dec 2020,Catalonia,15.4,13.8,17.0
2,A,Jun 2020,Italy,50.1,47.6,52.6
3,A,Nov/Dec 2020,Italy,30.1,27.1,33.1
4,A,Jun 2020,UK,16.1,12.8,19.4


In [358]:
base = (
    alt.Chart(f3d)
    .encode(
        x=alt.X(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelFontSize=12,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_filter('datum.Category=="D"')
)
bars1 = (
    base.mark_bar(color=colors["eco-turquiose"], width=30, xOffset=-18, opacity=0.7)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                gridOpacity=0.7,
                title="%",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleFontSize=12,
                labelFontSize=12,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=23,
                titleY=20,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=4,
                format=".0f",
            ),
            scale=alt.Scale(domain=[0, 65]),
        )
    )
    .transform_filter('datum.Time=="Jun 2020"')
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"], width=30, xOffset=18, opacity=0.8)
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
        )
    )
    .transform_filter('datum.Time!="Jun 2020"')
)
line1 = bars1.mark_bar(
    xOffset=-18,
    width=2,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
line2 = bars2.mark_bar(
    xOffset=18,
    width=2,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
    opacity=0.7,
).encode(y="Min:Q", y2="Max:Q")
text1 = bars1.mark_text(
    align="right",
    fontWeight="bold",
    dx=-21,
    dy=-8,
    color=scale_lightness(colors["eco-turquiose"], 0.8),
).encode(text="Value:N")
text2 = bars2.mark_text(
    align="left",
    fontWeight="bold",
    dx=+21,
    dy=-8,
    color=scale_lightness(colors["eco-light-blue"], 0.8),
).encode(text="Value:N")
l1 = (
    text1.mark_text(
        align="right",
        fontWeight="bold",
        dx=-21,
        dy=-24,
        color=scale_lightness(colors["eco-turquiose"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
l2 = (
    text2.mark_text(
        align="right",
        fontWeight="bold",
        dx=+42,
        dy=-24,
        color=scale_lightness(colors["eco-light-blue"], 0.8),
    )
    .encode(text="Time:N")
    .transform_filter('datum.Country=="Italy"')
)
layer1 = (
    (
        (bars1 + bars2 + line1 + line2 + text1 + text2 + l1 + l2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)